<a href="https://colab.research.google.com/github/holtnk/ASDLeads/blob/main/ASD_Lead_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To Run:

Windows: Ctrl + F9

Mac: Command + Ctrl + F9

Note: You may need to also press the fn key if your function row is multipurpose

Enter your zip code when prompted

In [ ]:
pip install requests


In [ ]:
pip install geopy


In [ ]:
pip install pandas

In [16]:
import requests
import csv
from geopy.geocoders import Nominatim
from geopy.distance import geodesic

taxonomy_search_list = [
    'Clinical Neuropsychologist',
    'Psychologist',
    'Cognitive & Behavioral Psychologist',
    'Clinical Psychologist',
    'Clinical Child & Adolescent Psychologist',
    'Family Psychologist',
    'Health Psychologist',
    'Health Service Psychologist',
    'Intellectual & Developmental Disabilities Psychologist',
    'Prescribing (Medical) Psychologist',
    'Psychoanalysis Psychologist',
    'School Psychologist',
    'Child & Adolescent Psychiatric/Mental Health Registered Nurse Behavioral Neurology & Neuropsychiatry Physician',
    'Diagnostic Neuroimaging (Psychiatry & Neurology) Physician',
    'Neurology with Special Qualifications in Child Neurology Physici',
    'Psychiatry Physician',
    'Child & Adolescent Psychiatry Physician',
    'Adolescent and Children Mental Health Clinic/Center',
    "Children's Hospital",
    'Child & Adolescent Psychiatric/Mental Health Clinical Nurse Spe Child & Family Psychiatric/Mental Health Clinical Nurse Specialist'
]

RADIUS = 100


def collect_leads(zip_code):
    with open('Leads.csv', 'a', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['Contact Name', 'NPI', 'Address', 'City', 'State', 'Zip', 'Phone', 'Taxonomy Codes', 'Taxonomy Description']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        # Check if file is empty to decide whether to write header
        csvfile.seek(0, 2)  # Go to end of file
        if csvfile.tell() == 0:  # Check file size
            writer.writeheader()  # Write header only if file is empty

        # Initialize a list to store taxonomy_desc values that do not return valid leads
        invalid_taxonomies = []

        # Load taxonomy descriptions
        for taxonomy_desc in taxonomy_search_list:
            providers = search_providers_by_zip(zip_code, taxonomy_desc)
            if providers:
                leads = []
                for provider in providers:
                    lead = {
                        'Contact Name': provider.get('basic', {}).get('name'),
                        'NPI': provider.get('number'),
                        'Address': provider.get('addresses', [{}])[0].get('address_1'),
                        'City': provider.get('addresses', [{}])[0].get('city'),
                        'State': provider.get('addresses', [{}])[0].get('state'),
                        'Zip': provider.get('addresses', [{}])[0].get('postal_code'),
                        'Phone': provider.get('addresses', [{}])[0].get('telephone_number'),
                        'Taxonomy Codes': ', '.join([taxonomy.get('code') for taxonomy in provider.get('taxonomies', [])]),
                        'Taxonomy Description': ', '.join([taxonomy.get('desc') for taxonomy in provider.get('taxonomies', [])])
                    }
                    if is_within_radius(zip_code, lead["Zip"]):
                        leads.append(lead)

                # Write leads to CSV
                if leads:
                    writer.writerows(leads)
                    print(f"Added leads for {taxonomy_desc}")
                else:
                    invalid_taxonomies.append(taxonomy_desc)
                    print(f"No leads found for {taxonomy_desc}")
            else:
                invalid_taxonomies.append(taxonomy_desc)
                print(f"No data found for {taxonomy_desc}")

        # Print or further process invalid taxonomy descriptions
        if invalid_taxonomies:
            print("\nTaxonomy descriptions without valid leads:")
            for desc in invalid_taxonomies:
                print(desc)
            print("\nPlease check these for proper spelling/classification and try again. Note that word order, abbreviations, and/or capitalizations may play a factor.")


def search_providers_by_zip(zip_code, taxonomy_desc):
    base_url = 'https://npiregistry.cms.hhs.gov/api/'
    url = f'{base_url}?postal_code={zip_code}&limit=200&taxonomy_description={taxonomy_desc}&version=2.1'
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        results = data.get('results', [])
        return results
    else:
        print(f'Error fetching data. Status code: {response.status_code}')
        return None


def is_within_radius(origin_zip_code, to_zip_code):
    geolocator = Nominatim(user_agent="zipcode_distance_calculator")

    # Isolate to the first 5 digits of the zip code
    origin_zip_code = origin_zip_code[:5]
    to_zip_code = to_zip_code[:5]

    origin = geolocator.geocode(f"{origin_zip_code}, USA")
    destination = geolocator.geocode(f"{to_zip_code}, USA")

    if not origin:
        raise ValueError(f"Cannot find location for zip code {origin_zip_code}")
    if not destination:
        raise ValueError(f"Cannot find location for zip code {to_zip_code}")

    distance = geodesic((origin.latitude, origin.longitude), (destination.latitude, destination.longitude)).miles
    return distance <= RADIUS


if __name__ == "__main__":
    zip_code = input("Enter your zip code: ")
    print(f"Searching for providers within {RADIUS} miles of {zip_code}")
    collect_leads(zip_code)
    print("Search completed")

Enter your zip code: 98144
Searching for providers within 100 miles of 98144
Added leads for Clinical Neuropsychologist
Added leads for Psychologist
No data found for Cognitive & Behavioral Psychologist
No data found for Clinical Psychologist
Added leads for Clinical Child & Adolescent Psychologist
No data found for Family Psychologist
No data found for Health Psychologist
No data found for Health Service Psychologist
Added leads for Intellectual & Developmental Disabilities Psychologist
No data found for Prescribing (Medical) Psychologist
No data found for Psychoanalysis Psychologist
No data found for School Psychologist
No data found for Child & Adolescent Psychiatric/Mental Health Registered Nurse Behavioral Neurology & Neuropsychiatry Physician
No data found for Diagnostic Neuroimaging (Psychiatry & Neurology) Physician
No data found for Neurology with Special Qualifications in Child Neurology Physici
No data found for Psychiatry Physician
No data found for Child & Adolescent Psych